<h1>Richardson-Lucy Deconvolution/Imaging Using COSI-Balloon Flight Data: Crab </h1>

***

## Import packages
We will need to import the cosipy-classic functions from COSIpy_dc1.py, response_dc1, and COSIpy_tools_dc1, as well as some other standard Python packages

In [ ]:
from COSIpy_dc1 import *
import response_dc1
from COSIpy_tools_dc1 import *
import pickle
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from tqdm.autonotebook import tqdm as tqdm

<h3>Define input data and response:</h3>

In [ ]:
data_dir = '../../data_products' # directory containing data & response files
filename = 'Crab_COSIBalloonData.tra.gz' # combined simulation
response_filename = data_dir + '/Continuum_imaging_response.npz' # detector response

<h3>Reading in data and define analysis object</h3>

In [ ]:
analysis = COSIpy(data_dir,filename)
analysis.read_COSI_DataSet()

<h3>Bin the data</h3>

In [ ]:
#Define the bin sizes
Delta_T = 1800 #30 min
energy_bin_edges=np.array([150,  220,  325,  480,  520,  765, 1120, 1650, 2350, 3450, 5000])
pixel_size = 6.

analysis.dataset.time_binning_tags(time_bin_size=Delta_T) # time binning
analysis.dataset.init_binning(energy_bin_edges=energy_bin_edges,pixel_size=pixel_size) # energy and pixel binning
analysis.dataset.get_binned_data() # bin data

In [ ]:
# Total exposure time of the Crab for this selected data set
print('Total Crab exposure in seconds:', analysis.dataset.times.total_time)

<h3>Plot a raw-spectrum and light curve<\h3>

In [ ]:
analysis.dataset.plot_raw_spectrum()
plt.xscale('log')

analysis.dataset.plot_lightcurve()

<h3> Select which energy bin to use for the following imaging </h3>

In [ ]:
# ebin 2 is recommended because it has the highest effective area and flux
# ebin:             0    |      1    |    2    |    3    |    4     |     5    |     6     |     7     |     8    |     9    |
# energies(keV): 150-220 | 220 - 325 | 325-480 | 480-520 | 520-765 | 765-1120 | 1120-1650 | 1650-2350 | 2350-3450| 3450-5000 |

#ebin, DeltaE = 0, 220 - 150
#ebin, DeltaE = 1, 325 - 220
ebin, DeltaE = 2, 480 - 325
#ebin, DeltaE = 4, 765 - 520
#ebin, DeltaE = 5, 1120 - 765

<h3>Define the pointing object with the cosipy pointing class</h3>

In [ ]:
# Definition of poitings / pointing object (balloon location + Earth rotation)
pointing = Pointing(dataset=analysis.dataset,)

<h3>Visualize the path of the Crab through the field-of-view</h3>

In [ ]:
# Crab Galactic coordinates (long, lat)
l1,b1 = 184.55746, -5.78436

In [ ]:
# Plot the position of the Crab during flight, relative to the zenith of the detector
plt.figure(figsize=(15,8))
l_exp = pointing.zpoins[:,0]
b_exp = pointing.zpoins[:,1]
l_exp[l_exp > 0] -= 360
plt.plot(l_exp,b_exp,'o')
plt.plot(l1-360,b1,'*b',markersize=20, label='crab')
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.title('COSI Instrument Pointings')
plt.legend()

In [ ]:
# Plot elevation angle of Crab relative to instrument zenith
analysis.plot_elevation([l1],[b1],['Crab'])

<h3>Define the BG model</h3>

Due to the fact that flight data background is highly variable, we use a background tracer to better track the changes for improved background modeling

In [ ]:
# Define background tracer
tracer = np.sum(analysis.dataset.binned_data,axis=(1,2,3))
tracer = tracer/np.mean(tracer)

For the background model we can select from models derived from different simulations of the 2016 flight, or a model derived from balloon data. The 'default 6deg' model is derived from the 2016 balloon data. This means we're using the flight data to define a background response.

In [ ]:
# background object defined with background tracer (making the zeros zero)
background = BG(dataset=analysis.dataset,mode='default 6deg',tracer=tracer)

<h3>Reading in the continuum detector response for the full 2016 flight:</h3>

In [ ]:
rsp = response_dc1.SkyResponse(filename=response_filename, pixel_size=6)

In [ ]:
# Print response dimensions
print('response dimensions (lat, lon, phi, psi/chi, energy bins):', rsp.rsp.response_grid_normed_efinal.shape)

What follows are some hard-coded sections where we define the sky and background:

<h3>Defining the sky in pixels to make images:</h3>

In [ ]:
# Defining our sky-grid on a regular 6x6 pixel grid
binsize = 6.
# Set l and b pixel edges
l_arrg = np.linspace(-180,180,int(360/binsize)+1)
b_arrg = np.linspace(-90,90,int(180/binsize)+1)
# Set number of pixels in l and b
n_l = int(360/binsize)
n_b = int(180/binsize)
# Creating a grid
L_ARRg, B_ARRg = np.meshgrid(l_arrg,b_arrg)
# Choosing the center points as representative
l_arr = l_arrg[0:-1]+binsize/2
b_arr = b_arrg[0:-1]+binsize/2
L_ARR, B_ARR = np.meshgrid(l_arr,b_arr)

# Define solid angle for each pixel for normalisations later
deg2rad = np.pi/180.
domega = (binsize*deg2rad)*(np.sin(np.deg2rad(B_ARR+binsize/2)) - np.sin(np.deg2rad(B_ARR-binsize/2)))

<h3>Convert sky grid to zenith/azimuth pairs for all pointings:</h3>

In [ ]:
# Calculate the zeniths and azimuths on the defined grid for all times
zensgrid,azisgrid = zenaziGrid(pointing.ypoins[:,0],pointing.ypoins[:,1],
                               pointing.xpoins[:,0],pointing.xpoins[:,1],
                               pointing.zpoins[:,0],pointing.zpoins[:,1],
                               L_ARR.ravel(),B_ARR.ravel())
# Reshape for the following routines 
zensgrid = zensgrid.reshape(n_b,n_l,len(pointing.xpoins))
azisgrid = azisgrid.reshape(n_b,n_l,len(pointing.xpoins))

<h3>Getting the observation indices where we actually have measured photons (important for later):</h3>

In [ ]:
#Determine the int value of pixel indexes that have non-zero values
nonzero_idx = background.calc_this[ebin]
nonzero_idx_num = nonzero_idx.shape[0]

<h3>Below is a function created to obtain the response of an image for any arbitrary time binning:</h3>

In [ ]:
def get_image_response_from_pixelhit_general(Response,
                                             zenith,
                                             azimuth,
                                             dt,
                                             n_hours,
                                             binsize=6,
                                             cut=90,
                                             altitude_correction=False,
                                             al=None):
    """
    Get Compton response from hit pixel for each zenith/azimuth vector input.
    Binsize determines regular sky coordinate grid in degrees.

    :param: zenith        Zenith positions of all points of predefined sky grid with
                          respect to the instrument (in deg)
    :param: azimuth       Azimuth positions of all points of predefined sky grid with
                          respect to the instrument (in deg)
    :option: binsize      Default 5 deg (matching the sky dimension of the response). If set
                          differently, make sure it matches the sky dimension as otherwise,
                          false results may be returned
    :option: cut          Threshold to cut the response calculation after a certain zenith angle.
                          Default 60
    :param: n_hours       Number of hours in cdxervation
    :option: altitude_correction Default False: use interpolated transmission probability, normalised to 33 km and 500 keV,
                          to modify number of expected photons as a function of altitude and zenith angle of cdxervation
    :option: al           Altitude values according to dt from construct_pointings(); used of altitude_correction is set to True
    """

    # Azimuthal angle is periodic in the range [0,360]
    # Zenith angle ranges from [0,180]

    # Check which pixel (index) was hit on regular grid
    hit_pixel_zi = np.floor(zenith/binsize)
    hit_pixel_ai = np.floor(azimuth/binsize)

    # Check which pixel centre
    hit_pixel_z = (hit_pixel_zi+0.5)*binsize
    hit_pixel_a = (hit_pixel_ai+0.5)*binsize

    # Check which zeniths are beyond threshold
    bad_idx = np.where(hit_pixel_z > cut)

    # Choose pixels with hits to write to output array
    za_idx = np.array([hit_pixel_zi,hit_pixel_ai]).astype(int)

    nz = zenith.shape[2]

    n_lon = int(360/binsize)
    n_lat = int(180/binsize)
    
    l_arrg = np.linspace(-180,180,int(360/binsize)+1)
    b_arrg = np.linspace(-90,90,int(180/binsize)+1)
    L_ARRg, B_ARRg = np.meshgrid(l_arrg,b_arrg)
    l_arr = l_arrg[0:-1]+binsize/2
    b_arr = b_arrg[0:-1]+binsize/2
    L_ARR, B_ARR = np.meshgrid(l_arr,b_arr)

    # Take care of regular grid by applying weighting with latitude
    weights = ((binsize*np.pi/180)*(np.sin(np.deg2rad(B_ARR+binsize/2)) - np.sin(np.deg2rad(B_ARR-binsize/2)))).repeat(nz).reshape(n_lat,n_lon,nz)
    weights[bad_idx] = 0

    
    # Check for negative weights and indices and remove them
    weights[za_idx[0,:] < 0] = 0.
    weights[za_idx[1,:] < 0] = 0.
    za_idx[0,za_idx[0,:] < 0] = 0.
    za_idx[1,za_idx[1,:] < 0] = 0.
    
    
    if altitude_correction == True:
        altitude_response = return_altitude_response()
    else:
        altitude_response = one_func

    # Get responses for each pixel    
    image_response = np.zeros((n_hours,n_lat,n_lon,Response.shape[2]))

    for c in tqdm(range(n_hours)):
        cdx = np.where((pointing.cdtpoins > analysis.dataset.times.times_min[analysis.dataset.times.n_ph_dx[c]]) &
                       (pointing.cdtpoins <= analysis.dataset.times.times_max[analysis.dataset.times.n_ph_dx[c]]))[0]
        
        # This calculation is a look-up of the response entries. 
        image_response[c,:,:,:] += np.sum(Response[za_idx[0,:,:,cdx],za_idx[1,:,:,cdx],:]*np.einsum('klij->iklj', weights[:,:,cdx,None])*dt[cdx,None,None,None],axis=0)

    return image_response

<h3>Translating the sponse in the Compton Data Space coordinates to the sky</h3>

In [ ]:
sky_response_CDS = rsp.rsp.response_grid_normed_efinal.reshape(
    n_b,
    n_l,
    analysis.dataset.phis.n_phi_bins*\
    analysis.dataset.fisbels.n_fisbel_bins,
    10)[:,:,nonzero_idx,ebin]

In [ ]:
# reduced response dimensions:
# lat x lon x CDS
sky_response_CDS.shape

<h3>Calculation of the general response for the current data set</h3>

In [ ]:
sky_response_scaled = get_image_response_from_pixelhit_general(
    Response=sky_response_CDS,
    zenith=zensgrid,
    azimuth=azisgrid,
    dt=pointing.dtpoins,
    n_hours=analysis.dataset.times.n_ph,
    binsize=6.,
    #cut=90.,
    altitude_correction=False,
    al=np.ones(len(pointing.dtpoins)))

In [ ]:
# data-set-specific response dimensions
# times x lat x lon x CDS

print('The data specific sky response dimensions:',sky_response_scaled.shape)
sky_response_scaled_time = sky_response_scaled.shape[0]

<h3>Calculation of the 'exposure map' - the response weighted by time:</h3>

In [ ]:
expo_map_crab = np.zeros((n_b,n_l))
for i in tqdm(range(sky_response_scaled_time)):
    expo_map_crab += np.sum(sky_response_scaled[i,:,:,:],axis=2)

<h3> Choosing region of bad exposure - not seen by the instrument during the selected observation time:</h3>

In [ ]:
bad_expo = np.where(expo_map_crab/domega <= 100)

<h3>Plotting the exposure map weighted with the pixel size:</h3>

In [ ]:
plt.figure(figsize=(8,8))
plt.style.use('default')
plt.subplot(projection='aitoff')
plt.pcolormesh(L_ARRg*deg2rad,B_ARRg*deg2rad,np.flip(expo_map_crab/domega,axis=1))
plt.colorbar(orientation='horizontal')
cont1 = plt.contour(L_ARR*deg2rad,B_ARR*deg2rad,np.flip(expo_map_crab/domega,axis=1),colors='green')
cont2 = plt.contour(L_ARR*deg2rad,B_ARR*deg2rad, np.flip(expo_map_crab/domega,axis=1), levels =[0],colors='r')
plt.xticks(np.array([-150,-120,-90,-60,-30,0,30,60,90,120,150])*deg2rad,labels=[r'$150^{\circ}$'+'\n',
                                                         r'$120^{\circ}$'+'\n',
                                                         r'$90^{\circ}$'+'\n',
                                                         r'$60^{\circ}$'+'\n',
                                                         r'$30^{\circ}$'+'\n',                       
                                                         r'$0^{\circ}$'+'\n',
                                                         r'$330^{\circ}$'+'\n',
                                                         r'$300^{\circ}$'+'\n',                       
                                                         r'$270^{\circ}$'+'\n',
                                                         r'$240^{\circ}$'+'\n',                                                    
                                                         r'$210^{\circ}$'+'\n'],color='lightgray')

plt.scatter(np.deg2rad(360-l1),np.deg2rad(b1),color='aquamarine',marker='.',s=500, label= 'Crab')
plt.legend()
plt.show()


<h1>Begin Richardson-Lucy Deconvolution:</h1>

<h3>Define function for a starting map fo the RL deconvolution - an isotropic map:</h3>

In [ ]:
def IsoMap(ll,bb,A0,binsize=6):
    shape = np.ones(ll.shape)
    norm = np.sum(shape*(binsize*np.pi/180)*(np.sin(np.deg2rad(bb+binsize/2)) - np.sin(np.deg2rad(bb-binsize/2))))
    val = A0*shape/norm
    return val

<h3> Define suitable background time nodes:</h3>
We select background time nodes generated from a Baysian Blocks analysis to determine optimal sections of data where each background section if fit independently.
For this imaging demonstration, we provide the optimal time nodes for this data set.

In [ ]:
# Selected background time notes [time bin]
all_edges = np.array([  0 ,   6,  12,  23, 35, 48,  63,  79,  93,
                       107, 121, 134, 149, 163, 176, 187, 199, 209,
                       221, 231, 243, 255, 269, 281, 293, 308])
all_edges_num = all_edges.shape[0]

In [ ]:
plt.figure(figsize=(15,5))
plt.step(np.arange(len(np.sum(analysis.dataset.binned_data,axis=(1,2,3)))),np.sum(analysis.dataset.binned_data,axis=(1,2,3)))
for e in all_edges:
    plt.axvline(e,linestyle='-', linewidth=0.5)
    plt.title("Baysian Analysis Background Time Cuts")
    plt.xlabel("Time Bins [30min/bin]")
    plt.ylabel("Counts")

<h3>Add these time nodes to the current background model:</h3>

In [ ]:
background.make_bg_cuts(list(all_edges+1))

<h3>Select only the previously defined energy bin for data set:</h3>

In [ ]:
crab_dataset = analysis.dataset.binned_data[:,ebin,:,:].reshape(sky_response_scaled_time,30*1145)[:,nonzero_idx]

<h3>Select this energy bin for the background model and setting the cut values, indices, and total number of background cuts applied:</h3>

In [ ]:
background_model = background.bg_model_reduced[ebin]

In [ ]:
bg_cuts, idx_arr, Ncuts = background.bg_cuts, background.idx_arr, background.Ncuts

<h3> Set the initial background fits in the Richardson-Lucy algorithm to 1 for the first iteration, then to random values between 0.83 and 0.93 for the following iteration</h3>

In [ ]:
fitted_bg_it1 = np.full(np.shape(all_edges), 1.0)
fitted_bg = 0.85+np.random.rand(all_edges_num)*0.1

<h1>Richardson-Lucy algorithm (individual step explaination included in code):</h1>

In [ ]:
# Initial map (isotropic flat, small value)
map_init = IsoMap(L_ARR,B_ARR,1)

# Define background 
bg_cuts, idx_arr, Ncuts = background.bg_cuts, background.idx_arr, background.Ncuts

# Number of RL iterations (100 iterations is recommended)
iterations = 10

# Scalling factor for the 'Delta map', values of either 1000 or 2000 are recommended
afl_scl = 1000.

# Below are several initialise arrays to save images and other parameters to maps per iteration
map_iterations = np.zeros((n_b,n_l,iterations))
# Likelihood of maps (vs. intitial map which is fully background)
map_likelihoods = np.zeros(iterations)
# Fit likelihoods
intermediate_lp = np.zeros(iterations)
# Acceleration parameters
acc_par = np.zeros(iterations)
# Background parameters
bg_pars = np.zeros((iterations,Ncuts))

# As the zeroth iteration, copy initial map to become the 'old map' (see below)
map_old = map_init
# cf. Knoedlseder+1997 what the values denominator etc are
denominator = expo_map_crab

# The zeroth iteration is the initial map
map_iterations[:,:,0] = map_old

# Convolve this map with the response
expectation_init = 0
print('Convolving with response (init expectation), iteration 0')
for i in tqdm(range(n_b)):
    for j in range(n_l):
        expectation_init += sky_response_scaled[:,i,j,:]*map_init[i,j]

# Set the old expectation (in data space bins) to new expectation (convolved image)
expectation_old = expectation_init

# Run the modified RL algorithm for the number of iterations defined above
for its in tqdm(range(1,iterations)):
    
    # Setting the map to zero where we selected a bad exposure 
    map_old[bad_expo[0],bad_expo[1]] = 0
    # Check for each pixel to be finite
    map_old[np.where(np.isnan(map_old) == True)] = 0
    
    # Make new background for the next iteration
    bg_cuts, idx_arr, Ncuts = background.bg_cuts, background.idx_arr, background.Ncuts
    
    # Temporary background model
    tmp_model_bg = np.zeros((sky_response_scaled_time,background.bg_model_reduced[ebin].shape[1]))
    
    # Set the guess to the first background fit to 1 and set later fits to the random value of fitted_bg
    if its == 1:
        for g in range(sky_response_scaled_time):
            tmp_model_bg[g,:] = background_model[g,:]*fitted_bg_it1[idx_arr-1][g]
    else:
        for g in range(sky_response_scaled_time):
            tmp_model_bg[g,:] = background_model[g,:]*fitted_bg[idx_arr-1][g]


            
    # Expectation (in data space) is the image (expectation_old) plus the background (tmp_model_bg)
    expectation_tot_old = expectation_old + tmp_model_bg 

    # Calculate likelihood of currect total expectation
    map_likelihoods[its-1] = cashstat(crab_dataset.ravel(),expectation_tot_old.ravel())
    
    # Calculate numerator of RL algorithm
    numerator = 0
    print('Calculating Delta image, iteration '+str(its)+', numerator')
    for i in tqdm(range(sky_response_scaled_time)):
        for j in range(crab_dataset.shape[1]):
            numerator += (crab_dataset[i,j]/expectation_tot_old[i,j]-1)*sky_response_scaled[i,:,:,j]
    
    # Calculate delta map (denominator scaled by fourth root to avoid exposure edge effects)
    delta_map_tot_old = (numerator/denominator)*map_old*(denominator)**0.25
    
    # Check again for finite values and zero our bad exposure regions
    nan_idx = np.where(np.isnan(delta_map_tot_old) == 1)
    delta_map_tot_old[nan_idx[0],nan_idx[1]] = 0
    delta_map_tot_old[bad_expo[0],bad_expo[1]] = 0

    # These plots are not required but allow us to see how the RL algorithm is doing
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.pcolormesh(L_ARRg,B_ARRg,np.roll(map_old,axis=1,shift=30))
    plt.colorbar()
    plt.title('expectation old')
    plt.subplot(122)
    plt.pcolormesh(L_ARRg,B_ARRg,np.roll(delta_map_tot_old,axis=1,shift=30))
    plt.title('delta map')
    plt.colorbar()
    plt.show()
    
    # Convolve delta image
    print('Convolving Delta image, iteration '+str(its))
    conv_delta_map_tot = 0
    for i in tqdm(range(n_b)):
        for j in range(n_l):
            conv_delta_map_tot += sky_response_scaled[:,i,j,:]*delta_map_tot_old[i,j]
    
    # Find the maximum acceleration parameter to multiply with delta image
    # This will result in total image being positive everywhere
    print('Finding maximum acceleration parameter, iteration '+str(its))
    try:
        len_arr = []
        for i in range(0,10000):
            len_arr.append(len(np.where((map_old+delta_map_tot_old*i/afl_scl) < 0)[0]))
        len_arr = np.array(len_arr)
        afl = np.max(np.where(len_arr == 0)[0])
        print('Maximum acceleration parameter found: ',afl/afl_scl)

        
        # Fit delta map and current map to speed up RL algorithm
        print('Fitting delta-map in addition to old map, iteration '+str(its))
        # Dictionary for data set and prior
        data_multimap = dict(N = nonzero_idx_num,
                     Nh = sky_response_scaled_time,
                     Ncuts = Ncuts,
                     Nsky = 2,
                     acceleration_factor_limit=afl*0.95,
                     bg_cuts = bg_cuts,
                     bg_idx_arr = idx_arr,
                     y = crab_dataset.ravel().astype(int),
                     bg_model = tmp_model_bg,
                     conv_sky = np.concatenate([[expectation_old],[conv_delta_map_tot/afl_scl]]),
                     mu_flux = np.array([1,afl/2]),
                     sigma_flux = np.array([1e-2,afl]),
                     mu_Abg = fitted_bg,
                     sigma_Abg = np.repeat(0.1,Ncuts))

        # Fit;
        # Initial values for fit
        init = {}
        init['flux'] = np.array([1.,afl/2.])
        init['Abg'] = np.repeat(0.99,Ncuts)
        op2D = model_multimap.optimizing(data=data_multimap,init=init,as_vector=False,verbose=True,
                                                tol_rel_grad=1e3,tol_obj=1e-20)

        # Save values
        print('Saving new map, and fitted parameters, iteration '+str(its))
        intermediate_lp[its-1] = op2D['value']
        acc_par[its-1] = op2D['par']['flux'][1]
        bg_pars[its-1,:] = op2D['par']['Abg']
  
        # Make new map as old map plus scaled delta map
        map_new = map_old+op2D['par']['flux'][1]*delta_map_tot_old/afl_scl
    
        # Do the same with the expectation (data space)
        expectation_new = expectation_old + op2D['par']['flux'][1]*conv_delta_map_tot/afl_scl
        
    except:
        # If the fit failed...
        print('Proceeding without acceleration parameter.')
        map_new = map_old+delta_map_tot_old
        expectation_new = expectation_old + conv_delta_map_tot
    
    # Check finite values again
    if its == 1:
        bad_index_init = np.where(np.isnan(map_new) == True)
    
    # Also check that these values are zero
    map_new[bad_expo[0],bad_expo[1]] = 0
    map_new[np.where(np.isnan(map_new) == True)] = 0
    map_iterations[:,:,its] = map_new

    # Swap maps
    map_old = map_new
    
    # Also swap expectations
    expectation_old = expectation_new
    
    
    # Repeat for each iteration


<h3>Now we can extract/plot some results:</h3>

In [ ]:
# Calculate the significance of the last iteration
siglist = np.abs(map_likelihoods-map_likelihoods.max())
significance = math.sqrt(siglist[-2])
print("The significance (sigma) of this image is, ", significance)

<h3>Calculating the fluxes of the <em>total</em> maps<br>
    If flux at position is to be calculated, select central pixel and the neighbouring pixels around that.</h3>

In [ ]:
map_fluxesA = np.zeros(iterations)
for i in range(iterations):
    map_fluxesA[i] = np.sum(map_iterations[:,:,i]*domega)
print('Crab flux in ph/keV for this energy bin: ', map_fluxesA[iterations-1])

In [ ]:
map_fluxesB = np.zeros(iterations)
for i in range(iterations):
    #map_iterations_nan[:,:,iterations-1]/domega/analysis.dataset.times.total_time*(DeltaE)
    map_fluxesB[i] = np.sum(map_iterations[:,:,i-1]/analysis.dataset.times.total_time*(DeltaE))
print('Crab flux in ph/cm^2/s for this energy bin: ', map_fluxesB[iterations-1])

<h3>Plotting the likelihood ratio test of all the iterations:</h3>

In [ ]:
plt.plot(np.abs(map_likelihoods-map_likelihoods.max()),'o-')
plt.xlim(0,iterations-2 )
plt.ylim(100,750)
plt.xlabel('Iteration')
plt.ylabel('Test Statistic')

<h3> Plotting the flux of the image per iteration</h3>

In [ ]:
plt.plot(map_fluxesA,'o-')
plt.xlim(0,iterations)
plt.xlabel('Iteration')
plt.ylabel('Flux [ph/keV]')

<h3>Plot a map in galactic coordinates:</h3>

In [ ]:
map_iterations_nan = np.copy(map_iterations)

plt.figure(figsize=(10.24,7.68))

# bad exposure regions will be gray
cmap = plt.get_cmap('viridis')
cmap.set_bad('lightgray')
map_iterations_nan = np.copy(map_iterations)
bad_expo = np.where(expo_map_crab/domega < 100)
for i in range(iterations):
    map_iterations_nan[bad_expo[0],bad_expo[1],i] = np.nan

plt.subplot(projection='aitoff')
plt.pcolormesh(L_ARRg*deg2rad,B_ARRg*deg2rad,
               np.flip(map_iterations_nan[:,:,iterations-1]/domega,axis=1)/analysis.dataset.times.total_time*(DeltaE),
               cmap=cmap,
               norm=colors.PowerNorm(1),
               rasterized=True)
                
plt.xticks(np.array([-150,-120,-90,-60,-30,0,30,60,90,120,150])*deg2rad,labels=[r'$150^{\circ}$'+'\n',
                                                         r'$120^{\circ}$'+'\n',
                                                         r'$90^{\circ}$'+'\n',
                                                         r'$60^{\circ}$'+'\n',
                                                         r'$30^{\circ}$'+'\n',                       
                                                         r'$0^{\circ}$'+'\n',
                                                         r'$330^{\circ}$'+'\n',
                                                         r'$300^{\circ}$'+'\n',                       
                                                         r'$270^{\circ}$'+'\n',
                                                         r'$240^{\circ}$'+'\n',                                                    
                                                         r'$210^{\circ}$'+'\n'],color='lightgray')

plt.yticks(np.array([-60,-30,0,30,60])*deg2rad)

plt.xlabel('Gal. Lon. [deg]')
plt.ylabel('Gal. Lat. [deg]')
plt.grid()
cbar = plt.colorbar(orientation='horizontal', shrink=0.8)
cbar.set_label('Flux [ph cm$^{-2}$ s$^{-1}$]')

plt.scatter(np.deg2rad(360-l1),np.deg2rad(b1),color='aqua',marker='*',s=200, label= 'Crab')

plt.legend()